### CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

***

# Week 9: Further Image Processing

* Edge Detection
* Seam Carving

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Edge Detection

* We will use **convolution** with a suitable kernel to detect edges in a digital image.
* A so-called **Sobel Filter**  uses two $3 \times 3$ kernels $G_x$ and $G_y$, which are convolved with the original image.
* Their purpose is to identify horizontal and vertical changes:
  $$
  G_x = \left[\begin{array}{ccc}
  1 & 0 & -1 \\
  2 & 0 & -2 \\
  1 & 0 & -1
  \end{array}\right],
  \quad
  G_y = \left[\begin{array}{ccc}
  1 & 2 & 1 \\
  0 & 0 & 0 \\
  -1 & -2 & -1
  \end{array}\right],
  $$
* Note how $G_y = G_x^T$.
* Also, how $G_x = (1, 2, 1)^T (1, 0, -1)$ is a product of a **smoothing** filter and a **central difference**.
* As such, convolution with $G_x$ yields the $x$-part of the gradient of the image intensity function, and convolution with $G_y$ yields the $y$-part.
* The overall gradient of a matrix $A$ can this be found as $\sqrt{((A*G_x)^2 + (A*G_y)^2}$, where the square is meant component-wise (and not as matrix product).

* Let's start with the matrices:

In [ ]:
Gx = np.array([1,0,-1,2,0,-2,1,0,-1]).reshape((3,3))
Gy = np.array([1,2,1,0,0,0,-1,-2,-1]).reshape((3,3))
print(Gx,'\n\n',Gy)

In [ ]:
Gx = np.array([[1,2,1]]).T @ np.array([[1,0,-1]])  # matrix product
Gy = Gx.T   # matrix transpose
Gx, Gy

* Let's recreate last week's two circles image.

In [ ]:
expic = np.zeros((100,100,3),dtype='uint8')
for i in range(expic.shape[0]):
    for j in range(expic.shape[1]):
        if (i-20)**2 + (j-15)**2 <= 10**2:
            # a red circle with centre (20,15) and radius 10
            expic[i,j,0]=255
        if (i-60)**2 + (j-55)**2 <= 25**2:
            # a blue circle with centre (60,55) and radius 25
            expic[i,j,2] = 255

plt.imshow(expic)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(expic[40:50,30:40,:])

* Apply the two kernels

In [ ]:
xchange = np.zeros((100,100,3), dtype=float)
for r in range(1,99):
    for c in range(1,99):
        for k in range(3):
            val = 0
            for i in range(3):
                for j in range(3):
                    val = val + Gx[i, j]*float(expic[r-i+1, c-j+1, k])
            xchange[r,c,k] = val

In [ ]:
plt.imshow(np.uint8(255*xchange/xchange.max()))

In [ ]:
ychange = np.zeros((100,100,3), dtype=float)
for r in range(1,99):
    for c in range(1,99):
        for k in range(3):
            val = 0
            for i in range(3):
                for j in range(3):
                    val = val + Gy[i, j]*float(expic[r-i+1, c-j+1,k])
            ychange[r,c,k] = val

In [ ]:
plt.imshow(np.uint8(255*ychange/ychange.max()))

* Compute the gradient and check the 'blue' intensities.

In [ ]:
edges = np.sqrt(xchange**2 + ychange**2)

In [ ]:
edges.shape

In [ ]:
print(edges[40:50,30:40,2])

In [ ]:
plt.imshow(np.uint8(255*edges/edges.max()))

In [ ]:
pic = plt.imread("images/long_walk.png")

In [ ]:
plt.imshow(pic)

In [ ]:
pic.shape

In [ ]:
pic.dtype

In [ ]:
from PIL import Image

In [ ]:
expic = np.uint8(255*pic)
Image.fromarray(expic)

In [ ]:
xchange = np.zeros(expic.shape, dtype=float)
for r in range(1,200):
    for c in range(1,999):
        for k in range(3):
            val = 0
            for i in range(3):
                for j in range(3):
                    val = val + Gx[i, j]*float(expic[r-i+1, c-j+1, k])
            xchange[r,c,k] = val

In [ ]:
ychange = np.zeros(expic.shape, dtype=float)
for r in range(1,200):
    for c in range(1,999):
        for k in range(3):
            val = 0
            for i in range(3):
                for j in range(3):
                    val = val + Gy[i, j]*float(expic[r-i+1, c-j+1, k])
            ychange[r,c,k] = val

In [ ]:
edges = np.sqrt(xchange**2 + ychange**2)

In [ ]:
Image.fromarray(np.uint8(255*edges/edges.max()))

# Seam Carving

* Suppose a digital image is too wide.
* Clipping to fit will loose content on the side
* Scaling to size will distort content.
* Seam carving is a more subtle way of ... getting rid of one pixel per row.

In [ ]:
Image.fromarray(expic)

In [ ]:
expic.shape

* Here is the **clipped** version:

In [ ]:
Image.fromarray(expic[:,250:750])

* And here is the **scaled** version:

In [ ]:
Image.fromarray(expic[:,::2])

### Seams, Energy

* A **seam** is a connected string of pixels, one per row, starting with some pixel in the top row, such that the pixel in the next row is either immediately below the current pixel ($0$) or one to the left ($-1$) or one to the right ($+1$).
* The **energy** of a pixel is its gradient in the sense of edge detection.  Then **energy** of a seam is the sum of the energies of its pixels.
* **Aim:**  Find a seam of **minimal energy** and remove it form the picture ...

* Problem: There are just too many seams.  About $1000 \cdot 3^{200}$ in a $201 \times 1000$-array ...

In [ ]:
1000 * 3**200

* Here is a **recursive approach**:  Suppose `arr` is the matrix of energy values.
* Looking at the problem from row $r$ and column $c$, the minimal energy path from here consists of the $(r,c)$-pixel followed by the minimal energy path from either $(r+1, c-1)$ or $(r+1, c)$ or $(r+1,c+1)$, whichever has the smallest energy.
* Its energy is the energy of pixel $(r,c)$ plus the energy of that minimal energy path from one below.

In [ ]:
def minimise_from_pos(arr, r, c):
    if r == arr.shape[0] - 1:   # bottom row
        return arr[r,c], [c]
    minval, minpath = minimise_from_pos(arr, r+1, c)
    if c > 0:
        val, path = minimise_from_pos(arr, r+1, c-1)
        if val < minval:
            minval, minpath = val, path
    if c < arr.shape[1] - 1:
        val, path = minimise_from_pos(arr, r+1, c+1)
        if val < minval:
            minval, minpath = val, path
    return minval+arr[r,c], [c]+minpath
        

* Let's set up a random toy matrix `randarr` of integral energy values in `range(256)`.

In [ ]:
rng = np.random.default_rng()

In [ ]:
randarr = rng.integers(0, 256, size=(10,20))
print(randarr)
plt.imshow(randarr)

* Apply the function to the top left pixel.

In [ ]:
minimise_from_pos(randarr, 0, 0)

* Apply the function to **all** pixels in the top row.

In [ ]:
mins = [minimise_from_pos(randarr, 0, c) for c in range(randarr.shape[1])] 
mins

* Find the minimal energy value among these minimal energy seams.

In [ ]:
min(x[0] for x in mins)

* Find the **position** of the seam of minimal energy in that list.

In [ ]:
pos = np.argmin([x[0] for x in mins])
pos

In [ ]:
mins[pos]

* We could now procede and remove the corresponding pixels, one per row, from the image.
* However, this approach is very inefficient, as a lot of data is recomputed over and over again.  It probably won't work on arrays with more rows.
* In a more systematic approach, we try compute all that is needed only once, and store intermediate results.
* Here, it helps to construct two separate arrays, one that determines the paths, and one for the minimum energies.
* We take advantage of the fact that the bottom values are known.

* Starting from the bottom of the matrix, we will construct two arrays, `mins` and `minpaths` to compute the minimum weights/paths in each position.

In [ ]:
def minimise_paths(arr):
    
    rows, cols = arr.shape[0], arr.shape[1]
    mins = np.zeros(arr.shape)
    minpaths = np.zeros(arr.shape, dtype=int)

    # bottom row of mins is the same as bottom row of arr
    mins[-1,:] = arr[-1,:]

    # loop from bottom to top
    for r in reversed(range(rows-1)):
        for c in range(cols):
            minval = arr[r,c]+mins[r+1,c]
            minpaths[r,c] = 0
            if c > 0:
                val  = arr[r,c] + mins[r+1,c-1]
                if val < minval:
                    minval = val
                    minpaths[r,c] = -1
            if c < cols - 1:
                val = arr[r,c] + mins[r+1, c+1]
                if val < minval:
                    minval = val
                    minpaths[r,c] = 1
            mins[r,c] = minval
    return mins, minpaths
            

* Apply this to the toy matrix `randarr`.

In [ ]:
mins, paths = minimise_paths(randarr)
print(mins)

In [ ]:
print(paths)

In [ ]:
pos = np.argmin(mins[0])

In [ ]:
paths[:, pos]

* Does it also work for the `edges` matrix resulting from the edge detection on the larger image `expic`?

In [ ]:
edges.shape

* First, let's turn this color image into a proper 2D array, by adding up the three color intensities for each pixel

In [ ]:
sum_edges = edges.sum(axis=2)
sum_edges.shape

* Now apply the function to the resulting array `sum_edges`, perhaps excluding the first and last column from the search, as they are all-zero by construction.

In [ ]:
mins1, paths1 = minimise_paths(sum_edges[:,1:-1])
min(mins1[0]), max(mins1[0])

In [ ]:
pos = np.argmin(mins1[0])
pos

In [ ]:
mins1[0, pos]

In [ ]:
paths1[:, pos]

* Back to the toy matrix.
* The next step is to take the image, and the minimal paths computation, and remove those pixels from the image.
* This is achived by the followinfg function.

In [ ]:
def remove_path(im, minvals, minpaths):
    rows = im.shape[0]
    new = np.zeros(im.shape, dtype=np.uint8)[:, 1:]
    col = np.argmin(minvals[0])

    # loop through the rows
    for r in range(im.shape[0]):
        new[r, :col] = im[r, :col]
        new[r, col:] = im[r, col+1:]
        col = col + minpaths[r, col]
    return new


* Apply the function to the array `randarr` and its minimal energy seams.

In [ ]:
mins, paths = minimise_paths(randarr)
randarr2 = remove_path(randarr, mins, paths)

* Let's see the resulting reduced array `randarr2`:

In [ ]:
randarr2.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(randarr)
plt.subplot(1,2,2)
plt.imshow(randarr2)

* And do it again:

In [ ]:
mins, paths = minimise_paths(randarr2)
randarr3 = remove_path(randarr2, mins, paths)

In [ ]:
randarr3.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(randarr2)
plt.subplot(1,2,2)
plt.imshow(randarr3)

* ... and so on ...

## References

### Numpy

*  [`rng.integers`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.integers.html)
*  [`np.argmin`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)

### Other

* Edge Detection [[wikipedia]](https://en.wikipedia.org/wiki/Edge_detection)
* Sobel Filter [[wikipedia]](https://en.wikipedia.org/wiki/Sobel_operator)
* Seam Carving [[wikipedia]](https://en.wikipedia.org/wiki/Seam_carving)

##  Exercises

* Define a function `edginess` which takes a digital image as input, then applies the Sobel filter for edge detection and returns, as an array of the same size as the given image, the resulting gradient values.

* Apply the `edginess` function to the picture of the long walk.

* Define a function `seem_carved` which takes a digital images as input, computes its edginess in order apply the seam carving procedure and return a modified version of the given image which is one pixel less wide.

* Apply the seam carving procedure to the picture of the long walk sufficiently often to reduce its width by a factor $1/2$.